In [1]:
import tensornetwork as tn
import numpy as np
import matplotlib.pyplot as plt

In [363]:
import numpy as np

def initialize_state(n_qubits):
    """Initialize the quantum state to an equal superposition state."""
    state = np.ones((2,) * n_qubits) / np.sqrt(2**n_qubits)
    return state

def apply_single_qubit_gate(state, gate, target_qubit):
    """Apply a single-qubit gate to a tensor network state."""
    gate = gate.reshape((2, 2))
    # axes = list(range(len(state.shape)))
    # axes[target_qubit] = axes[-1]
    # axes[-1] = target_qubit
    state = np.tensordot(state, gate, axes=(target_qubit, 0))
    return np.moveaxis(state, -1, target_qubit)

# def apply_single_qubit_gate(state, gate, target_qubit):
#     """Apply a single-qubit gate to a tensor network state using matrix multiplication."""
#     n_qubits = len(state.shape)
#     # Move the target qubit to the first axis
#     state = np.moveaxis(state, target_qubit, 0)
#     # Reshape the state tensor to (2, 2^(n_qubits-1))
#     state = state.reshape(2, -1)
#     # Apply the gate using matrix multiplication
#     state = gate @ state
#     # Reshape back to the original shape
#     state = state.reshape((2,) + state.shape[1:])
#     # Move the target qubit back to its original position
#     state = np.moveaxis(state, 0, target_qubit)
#     return state

def apply_multi_controlled_z(state):
    """Apply a multi-controlled-Z gate to the tensor network state."""
    n_qubits = len(state.shape)
    # Multi-controlled-Z is equivalent to flipping the sign of the |11...1> state
    indices = tuple([1] * n_qubits)
    state[indices] *= -1
    return state

def apply_oracle(state, target_state):
    """Apply the oracle by flipping the phase of the target state."""
    shape = state.shape
    indices = tuple(int(b) for b in format(target_state, f'0{len(shape)}b'))
    state[indices] *= -1
    return state

def apply_diffusion(state):
    """Apply the diffusion operator."""
    n_qubits = len(state.shape)
    H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])

    # Apply H gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, H, qubit)
    print('Apply H gate to all qubits')
    print(state)

    # Apply X gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, X, qubit)
    print('Apply X gate to all qubits')
    print(state)

    # Apply multi-controlled-Z gate
    state = apply_multi_controlled_z(state)
    print(state)

    # Apply Z gate to 0th qubit
    state = apply_single_qubit_gate(state, Z, 0)
    print(state)

    # Apply X gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, X, qubit)
    print(state)

    # Apply Z gate to 0th qubit
    state = apply_single_qubit_gate(state, Z, 0)
    print(state)

    # Apply H gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, H, qubit)

    return state

def measure_state(state):
    """Measure the quantum state to find the most probable state."""
    probabilities = np.abs(state)**2
    most_probable_state = np.unravel_index(np.argmax(probabilities), state.shape)
    return most_probable_state

# Number of qubits
# n_qubits = 3

# # Initialize the quantum state
# state = initialize_state(n_qubits)

# # Target state to search for (in decimal representation)
# target_state = 5  # Corresponds to the binary state '101'

# # Number of Grover iterations
# num_iterations = int(np.floor(np.pi / 4 * np.sqrt(2**n_qubits)))

# # Perform Grover iterations
# for _ in range(num_iterations):
#     state = apply_oracle(state, target_state)
#     state = apply_diffusion(state)

# # Measure the final state
# result = measure_state(state)
# result_bin = ''.join(map(str, result))
# print("Measurement result: ", result_bin)


In [122]:
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
# a  = np.array([[.5, .5], [.5, -.5]])
a  = np.array([.5, -.5])
a
# a = apply_single_qubit_gate(a, H, 0)
# print(a)
# a = apply_single_qubit_gate(a, H, 0)
# print(a)

array([ 0.5, -0.5])

In [115]:
np.tensordot(np.tensordot(a, H, axes=(0, 0)), H, axes=(0,0))

array([ 0.5, -0.5])

In [110]:
apply_single_qubit_gate(a, H, 0)

array([0.        , 0.70710678])

In [101]:
(H @ a[0, :])

array([ 7.07106781e-01, -2.29934717e-17])

In [96]:
a = np.tensordot(a, H, axes=(0, 0))
a = np.tensordot(a, H, axes=(1, 0))
a

array([[ 0.5,  0.5],
       [ 0.5, -0.5]])

### start

In [364]:
state = initialize_state(2)
state

array([[0.5, 0.5],
       [0.5, 0.5]])

In [365]:
state = apply_oracle(state, 3)
state

array([[ 0.5,  0.5],
       [ 0.5, -0.5]])

In [362]:
state = apply_diffusion(state)
state

Apply H gate to all qubits
[[[ 5.00000000e-01  5.00000000e-01]
  [-1.11855716e-17 -1.11855716e-17]]

 [[ 5.00000000e-01  3.11164266e-19]
  [-2.74444113e-17  5.00000000e-01]]]
Apply X gate to all qubits
[[[ 5.00000000e-01  3.11164266e-19]
  [ 5.00000000e-01  5.00000000e-01]]

 [[-2.74444113e-17  5.00000000e-01]
  [-1.11855716e-17 -1.11855716e-17]]]
[[[ 5.00000000e-01  3.11164266e-19]
  [ 5.00000000e-01  5.00000000e-01]]

 [[-2.74444113e-17  5.00000000e-01]
  [-1.11855716e-17  1.11855716e-17]]]
[[[ 5.00000000e-01  2.74444113e-17]
  [ 3.11164266e-19 -5.00000000e-01]]

 [[ 5.00000000e-01  1.11855716e-17]
  [ 5.00000000e-01 -1.11855716e-17]]]
[[[ 5.00000000e-01  1.11855716e-17]
  [ 5.00000000e-01  2.74444113e-17]]

 [[ 5.00000000e-01 -1.11855716e-17]
  [ 3.11164266e-19 -5.00000000e-01]]]
[[[ 5.00000000e-01 -5.00000000e-01]
  [ 1.11855716e-17  1.11855716e-17]]

 [[ 5.00000000e-01 -3.11164266e-19]
  [ 2.74444113e-17  5.00000000e-01]]]


array([[[ 7.07106781e-01, -3.53553391e-01],
        [ 3.25176795e-17, -3.53553391e-01]],

       [[ 2.63607877e-17,  3.53553391e-01],
        [-6.73463194e-18, -3.53553391e-01]]])

In [368]:
np.kron(H, H) @ state.reshape(-1)
# np.kron(np.kron(H, H), H) @ state.reshape(-1)

array([ 0.5,  0.5,  0.5, -0.5])

In [371]:
H@H@ state

array([[ 0.5,  0.5],
       [ 0.5, -0.5]])

In [355]:
s2 = apply_single_qubit_gate(state, H, 0)
s2 = apply_single_qubit_gate(s2, H, 1)
# s2 = apply_single_qubit_gate(s2, H, 2)
print(s2)
np.moveaxis(s2, -1, 1)

[[[ 7.07106781e-01  3.53553391e-01]
  [ 2.29934717e-17  3.53553391e-01]]

 [[-1.58187870e-17  3.53553391e-01]
  [ 5.17222793e-35 -3.53553391e-01]]]


array([[[ 7.07106781e-01,  2.29934717e-17],
        [ 3.53553391e-01,  3.53553391e-01]],

       [[-1.58187870e-17,  5.17222793e-35],
        [ 3.53553391e-01, -3.53553391e-01]]])

In [358]:
s2 = np.tensordot(state, H, axes=(0, 0))
print(s2)
s2 = np.tensordot(s2, H, axes=(1, 0))
print(s2)

s2 = np.tensordot(s2, H, axes=(2, 0))
print(s2)

[[[ 5.00000000e-01 -1.11855716e-17]
  [ 5.00000000e-01 -1.11855716e-17]]

 [[ 5.00000000e-01 -1.11855716e-17]
  [-1.11855716e-17  5.00000000e-01]]]
[[[ 7.07106781e-01  2.29934717e-17]
  [-1.58187870e-17  5.17222793e-35]]

 [[ 3.53553391e-01  3.53553391e-01]
  [ 3.53553391e-01 -3.53553391e-01]]]
[[[ 5.00000000e-01  5.00000000e-01]
  [-1.11855716e-17 -1.11855716e-17]]

 [[ 5.00000000e-01  3.11164266e-19]
  [-2.74444113e-17  5.00000000e-01]]]


In [336]:
H @ state[0][1]

array([5.00000000e-01, 1.11855716e-17])

In [283]:
# np.kron(state[0, :], state[1, :])

In [299]:
state = apply_single_qubit_gate(state, H, 0)
print(state)
state = apply_single_qubit_gate(state, H, 1)
print(state)
state = apply_single_qubit_gate(state, H, 2)
print(state)

[[ 5.00000000e-01  5.00000000e-01  5.00000000e-01 -1.11855716e-17]
 [-1.11855716e-17 -1.11855716e-17 -1.11855716e-17  5.00000000e-01]]
[[ 7.07106781e-01  2.29934717e-17]
 [-1.58187870e-17  5.17222793e-35]
 [ 3.53553391e-01  3.53553391e-01]
 [ 3.53553391e-01 -3.53553391e-01]]


AxisError: source: axis 2 is out of bounds for array of dimension 2

In [233]:
apply_single_qubit_gate(state, X, 0)

array([[ 0.5, -0.5],
       [ 0.5,  0.5]])

In [246]:
state[:, 1]

array([ 0.5, -0.5])

In [245]:
X@state[:, 1]

array([-0.5,  0.5])

In [244]:
X@np.array([1, -1])

array([-1,  1])

In [222]:
state = apply_single_qubit_gate(state, X, 0)
print(state)
state = apply_single_qubit_gate(state, X, 1)
print(state)

[[ 0.5 -0.5]
 [ 0.5  0.5]]
[[-0.5  0.5]
 [ 0.5  0.5]]


In [223]:
state = apply_multi_controlled_z(state)
print(state)

[[-0.5  0.5]
 [ 0.5 -0.5]]


In [224]:
state = apply_single_qubit_gate(state, Z, 0)
print(state)

[[-0.5  0.5]
 [-0.5  0.5]]


In [225]:
state = apply_single_qubit_gate(state, X, 0)
print(state)
state = apply_single_qubit_gate(state, X, 1)
print(state)

[[-0.5  0.5]
 [-0.5  0.5]]
[[ 0.5 -0.5]
 [ 0.5 -0.5]]


In [226]:
state = apply_single_qubit_gate(state, Z, 0)
print(state)

[[ 0.5 -0.5]
 [-0.5  0.5]]


In [227]:
state = apply_single_qubit_gate(state, H, 0)
print(state)
state = apply_single_qubit_gate(state, H, 1)
print(state)

[[ 2.29934717e-17 -2.29934717e-17]
 [ 7.07106781e-01 -7.07106781e-01]]
[[5.97792087e-34 3.25176795e-17]
 [6.22328532e-19 1.00000000e+00]]


### done

In [ ]:


    # Apply Z gate to 0th qubit


    # Apply X gate to all qubits
    for qubit in range(n_qubits):
    state = apply_single_qubit_gate(state, X, qubit)
    print(state)

    # Apply Z gate to 0th qubit
    state = apply_single_qubit_gate(state, Z, 0)
    print(state)

    # Apply H gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, H, qubit)

In [132]:
a = np.array([-.25, -.25])
b = np.array([.25, -.25])

np.kron(a, b)

array([-0.0625,  0.0625, -0.0625,  0.0625])

In [134]:
a = np.array([1, 0])

H@H@a

array([ 1.00000000e+00, -2.23711432e-17])

In [137]:
state = apply_diffusion(state)
state

Apply H gate to all qubits
[[ 0.5  0.5]
 [ 0.5 -0.5]]
Apply X gate to all qubits
[[-0.5  0.5]
 [ 0.5  0.5]]
[[-0.5  0.5]
 [ 0.5 -0.5]]
[[-0.5  0.5]
 [-0.5  0.5]]
[[ 0.5 -0.5]
 [ 0.5 -0.5]]
[[ 0.5 -0.5]
 [-0.5  0.5]]


array([[5.97792087e-34, 3.25176795e-17],
       [6.22328532e-19, 1.00000000e+00]])

In [52]:
np.argmax(state.reshape(-1))

5